### Redes con tensorflow
#### Por Francisco Serradilla

#### Tareas

* [x] Entrenar el perceptrón multicapa suministrado.
* [x] Ampliarlo para meter más capas, usar activación relu y evaluar con un conjunto de test.
* Entrenar con algunos de los problemas suministrados.
* (Opcional) Probar otros optimizadores.
* (Opcional) Añadir dropout. ¿Mejora la generalización con alguno de los problemas suministrados?

In [19]:
# Creación de grafos

print('Loading tensorflow...')
import tensorflow as tf
print('Loaded')

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
# The value returned by the constructor represents the output of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix multiplication.
product = tf.matmul(matrix1, matrix2)

product

Loading tensorflow...
Loaded


<tf.Tensor 'MatMul_38:0' shape=(1, 1) dtype=float32>

In [20]:
# Ejecutar un grafo

# Launch the default graph.
sess = tf.Session()
# To run the matmul op we call the session 'run()' method, passing ‘product' which represents the output of the matmul op.  This indicates to the call that we want to get the output of the matmul op back.
# All inputs needed by the op are run automatically by the session.  They typically are run in parallel.
# The call 'run(product)' thus causes the execution of three ops in the graph: the two constants and matmul. The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[12.]]


In [21]:
# Variables

# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having launched the graph.  
init_op = tf.global_variables_initializer() # We first have to add the `init` Op to the graph

with tf.Session() as sess: # Launch the graph and run the ops
  sess.run(init_op) # Run the 'init' op
  print(sess.run(state)) # Print the initial value of 'state'
  for _ in range(3): # Run the op that updates 'state' and print 'state'.
    sess.run(update)
    print(sess.run(state))

0
1
2
3


In [22]:
# fetches

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed]) # fetches -> lo que se quiere obtener del cálculo
  print(result)

[21.0, 7.0]


In [23]:
# placeholders

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
  print(sess.run(output, feed_dict={input1:[7.], input2:[2.]}))

[14.]


In [24]:
# Carga ejemplos

import numpy as np

ni = 2 # número de entradas
nh = 10 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 5 #número de capas ocultas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/circulo.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]


In [25]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W1 = tf.Variable(tf.random_uniform([ni, nh], -0.1, 0.1, dtype=tf.float32))
b1 = tf.Variable(tf.random_uniform([nh], -0.1, 0.1, dtype=tf.float32))
W2 = tf.Variable(tf.random_uniform([nh, no], -0.1, 0.1, dtype=tf.float32))
b2 = tf.Variable(tf.random_uniform([no], -0.1, 0.1, dtype=tf.float32))

# aquí definimos la propagación de la red
s1 = tf.nn.sigmoid(tf.add(tf.matmul(e, W1), b1))
s = tf.nn.sigmoid(tf.add(tf.matmul(s1, W2), b2))

print("s1")
print(s1)

s1
Tensor("Sigmoid_2:0", shape=(?, 10), dtype=float32)


In [26]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s)), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s,1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    
sess.close()

Epoch: 100
RMS: 0.45541504
Accuracy: 0.48
Epoch: 200
RMS: 0.45538974
Accuracy: 0.48
Epoch: 300
RMS: 0.4211739
Accuracy: 0.6
Epoch: 400
RMS: 0.33847305
Accuracy: 0.84
Epoch: 500
RMS: 0.28175494
Accuracy: 0.84
Epoch: 600
RMS: 0.23962641
Accuracy: 1.0
Epoch: 700
RMS: 0.20809259
Accuracy: 1.0
Epoch: 800
RMS: 0.18143104
Accuracy: 1.0
Epoch: 900
RMS: 0.15534566
Accuracy: 1.0
Epoch: 1000
RMS: 0.12988131
Accuracy: 1.0


In [28]:
# Cargar datos morosos 

ni = 9 # número de entradas
nh = 3 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/morosos-ent.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]


In [34]:
# Cargar data2 class

ni = 2 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 2 #número de capas ocultas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/data_3classes_nonlinear_2D.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]



In [95]:
# Cargar datos quinielas

ni = 60 # número de entradas
nh = 60 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 25 #número de capas ocultas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/quinielas60-3-trn.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]

t = np.loadtxt("samples/quinielas60-3-tst.txt")
inputsT = t[:,:ni]
outputsT = t[:,ni:]

In [89]:
# Cargar datos aprobados 

ni = 3 # número de entradas
nh = 7 # número de neuronas en capa oculta
no = 1 # número de neuronas en capa de salida
capas = 10 #número de capas ocultas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/aprobado-ent.txt")
inputs = d[:,:ni]
outputs = d[:,ni:]


In [100]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])


# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))
for i in range (capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))



In [101]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()

Epoch: 10
RMS: 0.49809885
RMS test: 0.50498676
Accuracy: 0.5070707
Accuracy test: 0.4848485
Epoch: 20
RMS: 0.49731255
RMS test: 0.5050239
Accuracy: 0.5070707
Accuracy test: 0.4848485
Epoch: 30
RMS: 0.49543926
RMS test: 0.5024378
Accuracy: 0.5070707
Accuracy test: 0.4848485
Epoch: 40
RMS: 0.47584912
RMS test: 0.48363695
Accuracy: 0.5070707
Accuracy test: 0.4848485
Epoch: 50
RMS: 0.46552753
RMS test: 0.4653883
Accuracy: 0.5239057
Accuracy test: 0.5212121
Epoch: 60
RMS: 0.46071222
RMS test: 0.46091756
Accuracy: 0.5400673
Accuracy test: 0.53333336
Epoch: 70
RMS: 0.45522287
RMS test: 0.4595811
Accuracy: 0.55488217
Accuracy test: 0.53939396
Epoch: 80
RMS: 0.45063105
RMS test: 0.4578681
Accuracy: 0.5616162
Accuracy test: 0.55151516
Epoch: 90
RMS: 0.44623294
RMS test: 0.45731795
Accuracy: 0.56700337
Accuracy test: 0.56363636
Epoch: 100
RMS: 0.4409654
RMS test: 0.45949516
Accuracy: 0.5777778
Accuracy test: 0.53939396
Epoch: 110
RMS: 0.43548927
RMS test: 0.46013507
Accuracy: 0.5858586
Accuracy t

Epoch: 900
RMS: 0.25813106
RMS test: 0.60811067
Accuracy: 0.7616162
Accuracy test: 0.38787878
Epoch: 910
RMS: 0.30372742
RMS test: 0.5793199
Accuracy: 0.75824916
Accuracy test: 0.44848484
Epoch: 920
RMS: 0.25600186
RMS test: 0.58951324
Accuracy: 0.7616162
Accuracy test: 0.42424244
Epoch: 930
RMS: 0.24728979
RMS test: 0.58865476
Accuracy: 0.7622896
Accuracy test: 0.4181818
Epoch: 940
RMS: 0.2461796
RMS test: 0.5930342
Accuracy: 0.7622896
Accuracy test: 0.4121212
Epoch: 950
RMS: 0.2454771
RMS test: 0.59761983
Accuracy: 0.7622896
Accuracy test: 0.4
Epoch: 960
RMS: 0.24484348
RMS test: 0.5972571
Accuracy: 0.7622896
Accuracy test: 0.4
Epoch: 970
RMS: 0.2441579
RMS test: 0.6003781
Accuracy: 0.7622896
Accuracy test: 0.4060606
Epoch: 980
RMS: 0.24354582
RMS test: 0.6012243
Accuracy: 0.7622896
Accuracy test: 0.4
Epoch: 990
RMS: 0.2431193
RMS test: 0.5996692
Accuracy: 0.7622896
Accuracy test: 0.4060606
Epoch: 1000
RMS: 0.24266766
RMS test: 0.60077935
Accuracy: 0.7622896
Accuracy test: 0.4


In [106]:
# Creación de la red DROPOUT

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])

# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red con DROPOUT
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))

y = tf.placeholder("float", [None, 3])
keep_prob = tf.placeholder(tf.float32)  
drop_out = tf.nn.dropout(s[0], keep_prob) 
################## añadir a la misma capa o a la propagacion???????
s.append(tf.matmul(s[0], W[1]) + b[1]) #DROPOUT primera capa oculta

for i in range (1, capas - 1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))# Entrenamiento CON DROPOUT 


[<tf.Tensor 'Relu_419:0' shape=(?, 60) dtype=float32>, <tf.Tensor 'add_427:0' shape=(?, 60) dtype=float32>]


In [105]:
'''
RMS = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer,labels= y))
optimizer = tf.train.AdamOptimizer().minimize(RMS)
# definimos la función a minimizar (error cuadrático medio)

print("dropout")
print(drop_out)
print("out_layer")
print(out_layer)
print("cost")
print(cost)
'''
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s[capas-1])), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s[capas-1],1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print('RMS test:',sess.run(RMS, feed_dict={e: inputsT, d: outputsT}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy test:',sess.run(accuracy, feed_dict={e: inputsT, d: outputsT}))
    
sess.close()

Epoch: 100
RMS: 0.4466212
RMS test: 0.46445572
Accuracy: 0.5313131
Accuracy test: 0.4848485
Epoch: 200
RMS: 0.3521079
RMS test: 0.5064282
Accuracy: 0.6996633
Accuracy test: 0.43636364
Epoch: 300
RMS: 0.31860533
RMS test: 0.53900445
Accuracy: 0.7845118
Accuracy test: 0.43636364
Epoch: 400
RMS: 0.27480993
RMS test: 0.5931666
Accuracy: 0.8491582
Accuracy test: 0.35151514
Epoch: 500
RMS: 0.16324387
RMS test: 0.61194366
Accuracy: 0.94612795
Accuracy test: 0.36969697
Epoch: 600
RMS: 0.15360336
RMS test: 0.6156309
Accuracy: 0.9535354
Accuracy test: 0.35757574
Epoch: 700
RMS: 0.22684734
RMS test: 0.56864
Accuracy: 0.9144781
Accuracy test: 0.45454547
Epoch: 800
RMS: 0.14699294
RMS test: 0.6032701
Accuracy: 0.9535354
Accuracy test: 0.4060606
Epoch: 900
RMS: 0.15257205
RMS test: 0.59995925
Accuracy: 0.9535354
Accuracy test: 0.4181818
Epoch: 1000
RMS: 0.14896908
RMS test: 0.6117943
Accuracy: 0.94680136
Accuracy test: 0.4
